<a href="https://colab.research.google.com/github/robertoarturomc/ProgramacionConcurrente/blob/main/19_Concurrencia_con_asyncio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Programación Concurrente
## 19. Concurrencia con asyncio

`asyncio` nos permite escribir programas asincrónicos en Python, facilitando la gestión de tareas como:

* Servidores web
* Bots
* Aplicaciones concurrentes

In [2]:
import asyncio

async def decir_hola():
    print("¡Hola!")
    await asyncio.sleep(1)
    print("¡Adiós!")

# Llamar directamente con await (¡en una celda de Jupyter!)
await decir_hola()

¡Hola!
¡Adiós!


Aquí no trabajamos con Hilos/Procesos, sino con **corrutinas**.

`async def` define una corutina.

`await` pausa la ejecución hasta que se complete una operación.

Nota: estramos trabajando en un Notebook, por lo que estamos ejecutando nuestras corrutinas directamente con otro `await`. En un archivo ejecutable .py, lo haríamos con `asyncio.run()`

Por cierto, nota cómo usamos `asyncio.sleep()` en vez de `time.sleep()`. Esto es necesario para garantizar la correcta ejecución. Eso sí, al hacer requests, normalmente es necesario, para evitar ser marcado como un ataque DDoS.

### Genial, otra forma de hacer lo mismo de siempre, y ahora el profe me está hablando con abreviaturas que ni entiendo. ¿Al menos hay una ventaja al usar concurrencia asíncrona?

Veamos...

In [33]:
import urllib.request
import time

In [35]:
def descarga_secuencial(url, filename):
    urllib.request.urlretrieve(url, filename)

In [36]:
async def descarga_async(url, filename):
  urllib.request.urlretrieve(url, filename)

In [37]:
mi_url = 'https://example.com/'
mi_file = 'Ejemplo.html'
mi_file2 = 'Ejemplo2.html'

In [38]:
descarga_secuencial(mi_url, mi_file)

In [39]:
await descarga_async(mi_url, mi_file2)

In [46]:
start = time.time()

for i in range(5):
  descarga_secuencial(mi_url, mi_file)

print("Programa Secuencial Terminado")

end = time.time()

print("Se tardó ", end-start, "segundos")


Programa Secuencial Terminado
Se tardó  1.2345163822174072 segundos


In [48]:
start = time.time()

for i in range(5):
  await descarga_async(mi_url, mi_file2)

print("Programa Concurrente (async) Terminado")

end = time.time()

print("Se tardó ", end-start, "segundos")

Programa Concurrente (async) Terminado
Se tardó  1.215606689453125 segundos


Prácticamente lo mismo, ¿no? ¿En dónde está la ventaja?

In [49]:


async def main():
    await asyncio.gather(
      descarga_async(mi_url, 'Ejemplo2A.html'),
      descarga_async(mi_url, 'Ejemplo2B.html'),
      descarga_async(mi_url, 'Ejemplo2C.html'),
      descarga_async(mi_url, 'Ejemplo2D.html'),
      descarga_async(mi_url, 'Ejemplo2E.html'),
    )



In [52]:
# Va a fallar porque estamos en Notebook; en un ejecutable en Python, sí podría verse el ahorro

start = time.time()

asyncio.run( main() )

print("Programa Concurrente (async 2) Terminado")

end = time.time()

print("Se tardó ", end-start, "segundos")

RuntimeError: asyncio.run() cannot be called from a running event loop

Es decir, el ahorro real está cuando lo usamos para tareas de I/O, especialmente aquellas que implican acceder y escribir información remota.
Pero, como seguro lo recuerdan de su tarea (¿Verdad que todos leyeron?) aún no existe la capacidad directamente en Asyncio de trabajar con información Local de manera asíncrona.

### Profe, no sé cómo nadie se lo ha preguntado aún, pero...¿no es parecido a lo que hacíamos con Multihilos - threading ?

Sí, en un principio sí. Lo que cambia es lo que pasa por dentro:

1. Asyncio solo usa un hilo.
2. Asyncio es más "artesanal", te da más control.
3. Con asyncio las tareas no se paralelizan - solo cooperan entre sí.
4. Usa menos memoria.
5. Funciona mejor cuando son miles de tareas distintas - inicializar una corrutina es muy ligero.